### Times Now Scraper

In [1]:
from bs4 import BeautifulSoup
import requests

def times_now_scraper():
    URL="https://www.timesnownews.com/"
    r=requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')  
    heads={}
    count=0
    for row in soup.findAll('div', attrs = {'class':'component_1'}):
        count+=1
        head=row.text
        words=head.split(" ")
        head=""
        for i in words:
            if i.isalnum():
                head+=i+" "
            
        heads[head]={}
        heads[head]['Source']='Times_Now'
        heads[head]['link']=URL+row.a['href'][1:]
        
        if count==15:
            break
    row=soup.find('div', attrs = {'class':'component_4'})
    h=row.find('h2', attrs = {'class':'content'})
    #print(h)   
    head=h.text
    words=head.split(" ")
    head=""
    for i in words:
            if i.isalnum():
                head+=i+" "
            
    heads[head]={}
    heads[head]['Source']='Times_Now'
    heads[head]['link']=URL+row.a['href'][1:]
            
    count=0
    for row in soup.findAll('div', attrs = {'class':'component_3'}):
        count+=1
        head=row.text
        words=head.split(" ")
        head=""
        for i in words:
            if i.isalnum():
                head+=i+" "
            
        heads[head]={}
        heads[head]['Source']='Times_Now'
        heads[head]['link']=URL+row.a['href'][1:]
        
        if count==15:
                break
    return heads



In [2]:
from datetime import datetime
def extract_news_times(news):
    for n in news.keys():
        #print(n)
        link=news[n]['link']
        #print(link)
        r=requests.get(link)
        
        soup = BeautifulSoup(r.content, 'html5lib')
        Briefs=[]
        for row in soup.findAll('li', attrs = {'class':'mar-b10'}):
            texts=row.text
            text_ref=texts[1:len(texts)-1]+'.'
            Briefs.append(text_ref)
        if Briefs==[]:
            Briefs=[soup.find('h1').text]
        news[n]['gists']=Briefs
        
            
        row=soup.find('h1')
        news[n]['Titles']=[row.text]
        date=datetime.today().strftime('%Y-%m-%d')
        time=str(datetime.now().time())
        
        news[n]['Date']=date
        news[n]['Time']=time
        
    return news
        
        
        
        
        
        
        
    

### Republic TV scraper

In [3]:
def republic_tv_scraper():
    URL="https://www.republicworld.com/"
    r=requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')  
    heads={}
    count=0 
    
    for row in soup.findAll('div', attrs = {'class':'bdrTop-dddddd masthead-padding hover-effect'}):
        flag=0
        count+=1
        try:
            flag=1
            h=row.find('h2')
            head=h.text
            heads[head]={}
            heads[head]['Source']='Republic TV'
            heads[head]['link']=row.a['href']
        except:
            i=1
            
        if flag==0:
            try:
                h=row.find('h1')
                head=h.text
                heads[head]={}
                heads[head]['Source']='Republic TV'
                heads[head]['link']=row.a['href']
            except:
                i=1
        
        if count==15:
            break
    count=0      
    for row in soup.findAll('div', attrs = {'class':'bdrTop-dddddd hover-effect pad10 padtop10 padbtm10'}):
        count+=1
        flag=0
        try:
            flag=1
            h=row.find('h2')
            head=h.text
            heads[head]={}
            heads[head]['Source']='Republic TV'
            heads[head]['link']=row.a['href']
        except:
            i=1
        if flag==0:
            try:
                
                h=row.find('h1')
                head=h.text
                heads[head]={}
                heads[head]['Source']='Republic TV'
                heads[head]['link']=row.a['href']
            except:
                i=1
        
        if count==15:
            break
    count=0        
    for row in soup.findAll('div', attrs = {'class':'hover-effect'}):
        count+=1
        if count==20:
            break
        flag=0
        
        try:
            flag=1
            h=row.find('h2')
            head=h.text
            heads[head]={}
            heads[head]['Source']='Republic TV'
            heads[head]['link']=row.a['href']
            continue
        except:
            i=1
        try:
            h=row.find('h1')
            head=h.text
            heads[head]={}
            heads[head]['Source']='Republic TV'
            heads[head]['link']=row.a['href']
        except:
            i=1
        
        
            
    return heads
    
    
        
        
        
      

### Extractive Summarizer

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize 
def get_sentences(article):
  extracts=sent_tokenize(article)
  sentences=[]
  for extract in extracts:
    #print(extract)
    clean_sentence=extract.replace("[^a-zA-Z0-9]"," ")   ## Removing special characters
    #print(clean_sentence)
    obtained=word_tokenize(clean_sentence) 
    #print(obtained)
    sentences.append(obtained)

  return sentences



In [5]:

from nltk.cluster.util import cosine_distance
def get_similarity(sent_1,sent_2,stop_words):
  
  sent_1=[w.lower() for w in sent_1]
  sent_2=[w.lower() for w in sent_2]

  total=list(set(sent_1+sent_2)) ## Removing duplicate words in total set

  vec_1= [0] * len(total)
  vec_2= [0] * len(total)


  ## Count Vectorization of two sentences
  for w in sent_1:
    if w not in stop_words:
      vec_1[total.index(w)]+=1

  for w in sent_2:
    if w not in stop_words:
      vec_2[total.index(w)]+=1


  return 1-cosine_distance(vec_1,vec_2)


In [6]:
from nltk.corpus import stopwords
import numpy as np
def build_matrix(sentences):
  stop_words = stopwords.words('english')

  sim_matrix=np.zeros((len(sentences),len(sentences)))
  ## Adjacency matrix

  for id1 in range(len(sentences)):
    for id2 in range(len(sentences)):
      if id1==id2:  #escaping diagonal elements
        continue
      else:
        sim_matrix[id1][id2]=get_similarity(sentences[id1],sentences[id2],stop_words)

  return sim_matrix


In [7]:
def pagerank(text, eps=0.000001, d=0.85):
    score_mat = np.ones(len(text)) / len(text)
    delta=1
    ### iterative approach
    while delta>eps:
        score_mat_new = np.ones(len(text)) * (1 - d) / len(text) + d * text.T.dot(score_mat)
        delta = abs(score_mat_new - score_mat).sum()
        score_mat = score_mat_new

    return score_mat_new

In [8]:
def summarizer(article,req=3):
  summarized=[]

  sentence=get_sentences(article)

  sim_matrix=build_matrix(sentence)

  score=pagerank(sim_matrix)

  ranked_sentence = sorted(((score[i],s) for i,s in enumerate(sentence)), reverse=True)
  #print(ranked_sentence[2])
  
  for i in range(req):
      #print(ranked_sentence[i][1])
      summarized.append(" ".join(ranked_sentence[i][1]))

  return summarized



In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/abhijit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhijit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Republic text extractor

In [23]:
from datetime import datetime
def extract_news_rep(news):
    for n in news.keys():
        #print(n)
        link=news[n]['link']
        #print(link)
        r=requests.get(link)
        #print(link)
        
        soup = BeautifulSoup(r.content, 'html5lib')
        Briefs=[]
        sub=soup.find('div',attrs ={'class': 'storypicture'})
        
        head=soup.find('h1',attrs={'class': 'story-title'})
        #h=head.text.replace(',',' ')
        news[n]['Titles']=[head.text]
        sub=soup.find('div',attrs ={'class': 'flex flexResponsive flexJustifyBetween flexAlignItemsStart'})
        story=sub.find('div',attrs={'class': 'width100 storytext'})
        #print(story)
        Full=""
        for row in story.findAll('p'):
            texts=row.text
            Full+=texts+" "
        try:
            summary=summarizer(Full)
        except:
            summary=[head.text]
        #print(summary)
        news[n]['gists']=summary
        #news[n]['Titles']=row.text
        date=datetime.today().strftime('%Y-%m-%d')
        time=str(datetime.now().time())
        
        news[n]['Date']=date
        news[n]['Time']=time

    return news
        
        
        

### India Today Scraper

In [11]:
from bs4 import BeautifulSoup
import requests


def india_today_scraper():
    URL="https://www.indiatoday.in/"
    r=requests.get(URL)
    #print(r)
    soup = BeautifulSoup(r.content)  
    #print(soup)
    heads={}
    count=0
    row=soup.find('h2')
    link=row.a['href']
    head=row.text
    if URL not in link:
        link=URL+link[1:]
    heads[head]={}
    heads[head]['Source']='IndiaToday'
    heads[head]['link']=link
    #print(heads)
    rows=soup.findAll("div",attrs={'class': 'featured-post'})
    #print(rows)
    count=0
    for row in rows:
        if count==15:
            break
        count+=1
        link=row.a['href']
        head=row.text
        if URL not in link:
            link=URL+link[1:]
        
        heads[head]={}
        heads[head]['Source']='IndiaToday'
        heads[head]['link']=link
        
    sub=soup.find("div",attrs={'class': 'block block-itg-widget even'})
    rows=sub.findAll('li')
    count=0
    for row in rows:
        if count==15:
            break
        count+=1
        link=row.a['href']
        head=row.text
        if URL not in link:
            link=URL+link[1:]
            
        heads[head]={}
        heads[head]['Source']='IndiaToday'
        heads[head]['link']=link
        
        
    return heads
        
        
        
        
    
    
    
    
    
    

In [26]:
def extractor_it(news):
    for n in news.keys():
        #print(n)
        link=news[n]['link']
        #print(link)
        r=requests.get(link)
        #print(link)
        
        soup = BeautifulSoup(r.content, 'html5lib')
        Briefs=[]
        sub=soup.find("div",attrs={'class': 'story-section '})
        #print(sub)
        flag=0
        try:
            h=sub.find('h1')
            flag=1
            news[n]['Titles']=[h.text]
            s=[h.text]
        except:
            i=1
            
        if flag==0:
            try:
                flag=1
                h=sub.find('h2')
                news[n]['Titles']=[h.text]
                s=[h.text]
            except:
                i=1
          
        news[n]['Titles']=[n]
        s=[n]
        
        
        sub=soup.find("div",attrs={'class': 'story-left-section story-update'})
        #print(sub)
        Full=""
        try:
            rows=sub.findAll('p')
            for row in rows:
                texts=row.text
                #print(texts)
                Full+=texts+" "
            #print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
            summary=summarizer(Full,3)
        except:
            summary=s
        #print(link)
        #print(summary)
        news[n]['gists']=summary
        #news[n]['Titles']=row.text
        date=datetime.today().strftime('%Y-%m-%d')
        time=str(datetime.now().time())
        
        news[n]['Date']=date
        news[n]['Time']=time
        
    return news
        
        
    
        
        
    

### News 18 scraper

In [13]:
from bs4 import BeautifulSoup
import requests

def News_18_scraper():
    URL="https://www.news18.com/"
    r=requests.get(URL)
    #print(r)
    soup = BeautifulSoup(r.content,'html5lib')  
    #print(soup)
    heads={}
    sub= soup.find('div',attrs={'class': 'lead-story'})
    #print(sub)
    rows=sub.findAll('p')
    #print(rows)
    for row in rows:
        
        head=row.text
        heads[head]={}
        heads[head]['Source']='News18'
        #print(head)
        #print(row.a["href"])
        heads[head]['link']=row.a["href"]
        
    sub= soup.find('ul',attrs={'class': 'lead-mstory'})
    rows=sub.findAll('li')
    for row in rows:
        head=row.text
        heads[head]={}
        heads[head]["Source"]='News18'
        heads[head]["link"]=row.a["href"]
    
    return heads
    
        
    
    

In [14]:
def extractor_n18(news):
    for n in news.keys():
        #print(n)
        link=news[n]['link']
        
        r=requests.get(link)
        #print(link)
        
        soup = BeautifulSoup(r.content, 'html5lib')
        Briefs=[]
        #print(link)
        #print(soup)
        sub=soup.find("title")
        news[n]['Titles']=[sub.text]
        tit=sub.text
        flag=0
        try:
            flag=1
            text=""
            sub=soup.find('div',{'class':'lbcontent paragraph'})
            #print(sub)
            text+=sub.text+"\n"
            sub_2=soup.find('div',{'id':'article_body'})
            #print(sub_2.text)
            #print("++++++++++++++++++++++++")
            text+=sub_2.text
            summary=summarizer(text)
           #print(summary)
            #print(text)
        except:
            flag=0
            i=1
            
        if flag==0:
            text=""
            try:
                sub=soup.find('article',{'class':'article-content-box first_big_character'})
                rows=sub.findAll('p')
                for row in rows:
                    text+=row.text+"\n"
                summary=summarizer(text)
                    
            except:
                summary=tit
        #print(summary)
        news[n]['gists']=summary
        date=datetime.today().strftime('%Y-%m-%d')
        time=str(datetime.now().time())
        
        news[n]['Date']=date
        news[n]['Time']=time
        
    return news
        
                    
                
        
        

### File Creater

In [40]:
import pandas as pd
def Merge(dict1, dict2, dict3, dict4): 
    res = {**dict1, **dict2, **dict3, **dict4} 
    return res 

def file_creater(date):
    news_times=times_now_scraper()
    times_now=extract_news_times(news_times)
    news_rep=republic_tv_scraper()
    republic_tv=extract_news_rep(news_rep)
    news_it=india_today_scraper()
    india_today=extractor_it(news_it)
    n_18=News_18_scraper()
    News_18=extractor_n18(n_18)
    
    Merged=Merge(times_now,republic_tv,india_today,News_18)
    
    Merged_df=pd.DataFrame(Merged)
    Merged_df_final=Merged_df.transpose()
    df_final=Merged_df_final.reset_index()
    df_final_2=df_final.drop(['index'],axis=1)
    df_final_2.to_csv('feeds/Feed_'+date+'.csv',index=False)
    get_names('feeds/Feed_'+date+'.csv')
    
    return df_final_2
    
    
    
    
    
    

In [41]:
df=file_creater('08-09-2020')

/home/abhijit/.local/lib/python3.6/site-packages/numpy/core/_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [29]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [37]:
import os,ast
import pandas as pd
def get_names(file_name):    
    df=pd.read_csv(file_name)
    label=[]
    for j in range(len(df)):
        label_temp=[]
        try:
            title=ast.literal_eval(df.iloc[j]['Titles'])[0]
            doc=nlp(title)
            
            for X in doc.ents:
                if X.label_ not in ('ORDINAL','TIME','DATE','MONEY','PERCENT'):
                    label_temp.append(X.text)
        except:
            a=1
        label.append(label_temp)
    df['labels']=label
    
    df.to_csv(file_name,index=False)
            
                

In [39]:
get_names('feeds/Feed_07-09-2020.csv')